In [2]:
from ultralytics import YOLO
import cv2
import math
import json
import os

# Input video path and output JSON path
video_path_to_process = r'D:\FINAL PROJECT IG\project_maal\video\jcr.mp4'
output_json_path = 'hoinu1.json'

# Load YOLOv8 pose model
try:
    model = YOLO("yolov8s-pose.pt")
except Exception as e:
    print(f"Error loading YOLOv8s-pose.pt model: {e}")
    print("Please ensure the model file is available and accessible in the same directory.")
    exit()

# Normalize landmarks relative to shoulders
def normalize_landmarks(landmarks):
    if len(landmarks) < 7:
        print("Warning: Not enough landmarks (less than 7) for normalization. Skipping normalization.")
        return None
    try:
        if 5 not in range(len(landmarks)) or 6 not in range(len(landmarks)):
             print("Warning: Shoulder landmarks (index 5 or 6) are missing in the detected set. Cannot normalize.")
             return None
        left_shoulder = landmarks[5]
        right_shoulder = landmarks[6]
        if not all(k in left_shoulder for k in ['x', 'y']) or \
           not all(k in right_shoulder for k in ['x', 'y']):
            print("Warning: Shoulder landmark data incomplete (missing 'x' or 'y' keys). Cannot normalize.")
            return None
        center_x = (left_shoulder["x"] + right_shoulder["x"]) / 2
        center_y = (left_shoulder["y"] + right_shoulder["y"]) / 2
        scale = math.sqrt((left_shoulder["x"] - right_shoulder["x"]) ** 2 +
                          (left_shoulder["y"] - right_shoulder["y"]) ** 2)
        if scale == 0:
            scale = 1e-6
        normalized = []
        for lm in landmarks:
            if 'x' in lm and 'y' in lm:
                norm_x = (lm["x"] - center_x) / scale
                norm_y = (lm["y"] - center_y) / scale
                normalized.append({"x": norm_x, "y": norm_y})
            else:
                print(f"Warning: Skipping a landmark during normalization due to missing 'x' or 'y' data: {lm}")
                return None
        return normalized
    except Exception as e:
        print(f"An unexpected error occurred during normalization: {e}")
        return None

# Process video and save normalized landmarks to JSON
def process_video_for_landmarks(video_path, output_json_path, model):
    if not os.path.exists(video_path):
        print(f"Error: Video file not found at {video_path}")
        return
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return
    all_frames_data = []
    frame_count = 0
    print(f"\nProcessing video: {video_path}")
    while True:
        success, frame = cap.read()
        if not success:
            break
        frame_count += 1
        results = model(frame, verbose=False)[0]
        frame_data = {
            "frame": frame_count,
            "landmarks": []
        }
        if results.keypoints is not None and len(results.keypoints) > 0:
            best_person_keypoints_data = None
            max_keypoints = 0
            for person_kp_obj in results.keypoints:
                current_person_keypoints_data = person_kp_obj.data.cpu().numpy()
                if current_person_keypoints_data.ndim == 3 and current_person_keypoints_data.shape[0] == 1:
                    current_person_keypoints_data = current_person_keypoints_data[0]
                if len(current_person_keypoints_data) > max_keypoints:
                    max_keypoints = len(current_person_keypoints_data)
                    best_person_keypoints_data = current_person_keypoints_data
            if best_person_keypoints_data is not None:
                height, width = frame.shape[:2]
                current_frame_landmarks = []
                for id, kp in enumerate(best_person_keypoints_data):
                    x_norm = kp[0] / width
                    y_norm = kp[1] / height
                    current_frame_landmarks.append({
                        "id": id,
                        "x": x_norm,
                        "y": y_norm,
                        "visibility": kp[2]
                    })
                normalized_landmarks_for_frame = normalize_landmarks(current_frame_landmarks)
                if normalized_landmarks_for_frame:
                    frame_data["landmarks"] = normalized_landmarks_for_frame
                else:
                    print(f"Warning: Normalization failed for frame {frame_count} in {video_path}. Skipping landmarks for this frame.")
        all_frames_data.append(frame_data)
        if frame_count % 100 == 0:
            print(f"  Processed {frame_count} frames...")
    cap.release()
    with open(output_json_path, 'w') as f:
        json.dump(all_frames_data, f, indent=4)
    print(f"Finished processing {video_path}. Total frames saved: {len(all_frames_data)}")
    print(f"Pose coordinates saved to {output_json_path}")

# Start processing
process_video_for_landmarks(video_path_to_process, output_json_path, model)

print("\nReference JSON file generation complete. Please use 'hoinu1.json' with your main cheating detection script.")
print("Remember to also update the 'left_cheating_landmarks.json' if needed, using a separate video.")


100%|██████████| 22.4M/22.4M [00:09<00:00, 2.53MB/s]



Processing video: D:\FINAL PROJECT IG\project_maal\video\jcr.mp4
  Processed 100 frames...
  Processed 200 frames...
  Processed 300 frames...
Finished processing D:\FINAL PROJECT IG\project_maal\video\jcr.mp4. Total frames saved: 308
Pose coordinates saved to hoinu1.json

Reference JSON file generation complete. Please use 'hoinu1.json' with your main cheating detection script.
Remember to also update the 'left_cheating_landmarks.json' if needed, using a separate video.


In [4]:
from ultralytics import YOLO
import cv2
import math
import json
import os

# --- Configuration ---
video_path_to_process = r'D:\FINAL PROJECT IG\project_maal\video\jcl.mp4'  # Input video file path
output_json_path = 'hoinu2.json'  # Output JSON file for normalized pose landmarks

# --- Load YOLOv8 Pose Model ---
try:
    model = YOLO("yolov8s-pose.pt")
except Exception as e:
    print(f"Error loading YOLOv8s-pose.pt model: {e}")
    exit()

# --- Normalize landmarks using shoulder distance (scale + center) ---
def normalize_landmarks(landmarks):
    if len(landmarks) < 7:
        print("Warning: Not enough landmarks. Skipping normalization.")
        return None
    try:
        if 5 not in range(len(landmarks)) or 6 not in range(len(landmarks)):
            print("Warning: Missing shoulder landmarks.")
            return None

        left_shoulder = landmarks[5]
        right_shoulder = landmarks[6]

        if not all(k in left_shoulder for k in ['x', 'y']) or not all(k in right_shoulder for k in ['x', 'y']):
            print("Warning: Shoulder data incomplete.")
            return None

        center_x = (left_shoulder["x"] + right_shoulder["x"]) / 2
        center_y = (left_shoulder["y"] + right_shoulder["y"]) / 2

        scale = math.hypot(left_shoulder["x"] - right_shoulder["x"],
                           left_shoulder["y"] - right_shoulder["y"])
        if scale == 0:
            scale = 1e-6  # avoid division by zero

        normalized = []
        for lm in landmarks:
            if 'x' in lm and 'y' in lm:
                norm_x = (lm["x"] - center_x) / scale
                norm_y = (lm["y"] - center_y) / scale
                normalized.append({"x": norm_x, "y": norm_y})
            else:
                print(f"Warning: Landmark missing 'x' or 'y': {lm}")
                return None
        return normalized
    except Exception as e:
        print(f"Error during normalization: {e}")
        return None

# --- Process video and extract pose landmarks for most visible person ---
def process_video_for_landmarks(video_path, output_json_path, model):
    if not os.path.exists(video_path):
        print(f"Error: Video not found: {video_path}")
        return

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video: {video_path}")
        return

    all_frames_data = []
    frame_count = 0
    print(f"\nProcessing video: {video_path}")

    while True:
        success, frame = cap.read()
        if not success:
            break

        frame_count += 1
        results = model(frame, verbose=False)[0]

        frame_data = {
            "frame": frame_count,
            "landmarks": []
        }

        # If keypoints are detected
        if results.keypoints is not None and len(results.keypoints) > 0:
            best_person_keypoints_data = None
            max_keypoints = 0

            # Choose person with most keypoints
            for person_kp_obj in results.keypoints:
                current_person_keypoints_data = person_kp_obj.data.cpu().numpy()
                if current_person_keypoints_data.ndim == 3 and current_person_keypoints_data.shape[0] == 1:
                    current_person_keypoints_data = current_person_keypoints_data[0]

                if len(current_person_keypoints_data) > max_keypoints:
                    max_keypoints = len(current_person_keypoints_data)
                    best_person_keypoints_data = current_person_keypoints_data

            # Convert and normalize best keypoints
            if best_person_keypoints_data is not None:
                height, width = frame.shape[:2]
                current_frame_landmarks = []

                for id, kp in enumerate(best_person_keypoints_data):
                    x_norm = kp[0] / width
                    y_norm = kp[1] / height
                    current_frame_landmarks.append({
                        "id": id,
                        "x": x_norm,
                        "y": y_norm,
                        "visibility": kp[2]
                    })

                normalized_landmarks = normalize_landmarks(current_frame_landmarks)
                if normalized_landmarks:
                    frame_data["landmarks"] = normalized_landmarks
                else:
                    print(f"Warning: Normalization failed at frame {frame_count}")

        all_frames_data.append(frame_data)

        if frame_count % 100 == 0:
            print(f"  Processed {frame_count} frames...")

    cap.release()

    # Save all landmarks to JSON
    with open(output_json_path, 'w') as f:
        json.dump(all_frames_data, f, indent=4)

    print(f"Finished processing. Total frames: {len(all_frames_data)}")
    print(f"Saved to {output_json_path}")

# --- Run the processor ---
process_video_for_landmarks(video_path_to_process, output_json_path, model)

print("\nReference JSON file generation complete.")
print("Use 'hoinu2.json' in your main cheating detection script.")



Processing video: D:\FINAL PROJECT IG\project_maal\video\jcl.mp4
  Processed 100 frames...
  Processed 200 frames...
  Processed 300 frames...
Finished processing D:\FINAL PROJECT IG\project_maal\video\jcl.mp4. Total frames saved: 369
Pose coordinates saved to hoinu2.json

Reference JSON file generation complete. Please use 'hoinu2.json' with your main cheating detection script.
Remember to update the path in your main script to load 'hoinu2.json' for left turn landmarks.
